<b>HAM OR SPAM IN TENSORFLOW WITH COLAB</b><br>
--
AUTHOR : HAMORA HADI

In [ ]:

try:
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

In [ ]:
train_messages = pd.read_csv(TRAIN_DATA_URL, sep ='\t',names=["label", "message"])
test_messages = pd.read_csv(TEST_DATA_URL, sep ='\t',names=["label", "message"])

print("Size of training set:", train_messages.shape)
print("Size of test set:", test_messages.shape)

In [ ]:
train_messages.head(10)

In [ ]:
train_messages.describe()

HAM=0<br>
SPAM=1

In [ ]:
train_messages['msg_type']= train_messages['label'].map({'ham': 0, 'spam': 1})
train_msg = train_messages['message']
train_labels = train_messages['msg_type'].values

test_messages['msg_type']= test_messages['label'].map({'ham': 0, 'spam': 1})
test_msg = test_messages['message']
test_labels = test_messages['msg_type'].values

<b>TOKENIZATION</b>

In [ ]:
max_len = 50
trunc_type = "post" 
padding_type = "post" 
oov_tok = "<OOV>"  #out of vocabulary token will be added to word index in the corpus which is used to build the model. This is used to replace out of vocabulary words during text_to_sequence calls 
vocab_size = 700

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, char_level=False, oov_token = oov_tok)
tokenizer.fit_on_texts(train_msg)

**GET THE WORD INDEX!**

In [ ]:
word_index = tokenizer.word_index
word_index

<b>CHECK HOW MANY WORDS IN TRAINING DATA!</b>

In [ ]:
tot_words = len(word_index)
print('No. of unique tokens in training data: %s ' % tot_words)

**Sequencing and Padding**<br>


After tokenization, we represent each sentence by sequences of numbers using texts_to_sequences() from tokenizer object. Subsequently, we use pad_sequences() so that each sequence will have same length.

In [ ]:
training_seq = tokenizer.texts_to_sequences(train_msg)
training_padded = keras.preprocessing.sequence.pad_sequences(training_seq, maxlen=max_len, 
                                                             padding=padding_type, truncating=trunc_type)

test_seq = tokenizer.texts_to_sequences(test_msg)
test_padded = keras.preprocessing.sequence.pad_sequences(test_seq, maxlen=max_len, 
                                                             padding=padding_type, truncating=trunc_type)

In [ ]:
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', test_padded.shape)

<b>DENSE MODEL!</b>

In [ ]:
embedding_dim = 18

In [ ]:
from keras import layers

model_dense = tf.keras.Sequential([
  layers.Embedding(vocab_size, embedding_dim,input_length=max_len),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dense(24, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(1,activation='sigmoid')])

model_dense.summary()

In [ ]:
model_dense.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3) #stops training if validation performance doesn't improve after 3 epochs

history_dense = model_dense.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(test_padded,test_labels),callbacks=[early_stop], verbose=2)

In [ ]:
model_dense.evaluate(test_padded, test_labels)

In [ ]:
# Create a dataframe
metrics = pd.DataFrame(history_dense.history)
# Rename column
metrics.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy',
                         'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
def plot_graphs1(var1, var2, string):
    metrics[[var1, var2]].plot()
    plt.title('Dense Model: Training and Validation ' + string)
    plt.xlabel ('Number of epochs')
    plt.ylabel(string)
    plt.legend([var1, var2])
plot_graphs1('Training_Loss', 'Validation_Loss', 'loss')
plot_graphs1('Training_Accuracy', 'Validation_Accuracy', 'accuracy')

Now, create <b>LSTM</b>!

In [ ]:
model_LSTM = tf.keras.Sequential([
  layers.Embedding(vocab_size, embedding_dim,input_length=max_len),
  layers.LSTM(20, dropout=0.2,return_sequences=True),
  layers.LSTM(20, dropout=0.2,return_sequences=True),
  layers.Dense(1,activation='sigmoid')])

model_LSTM.summary()

In [ ]:
model_LSTM.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=2)

history_LSTM = model_LSTM.fit(training_padded, train_labels, epochs=num_epochs, validation_data=(test_padded,test_labels), callbacks=[early_stop], verbose=2)

In [ ]:
model_LSTM.evaluate(test_padded, test_labels)

In [ ]:
model_LSTM.evaluate(test_padded, test_labels)

In [ ]:

# Create a dataframe
metrics_LSTM = pd.DataFrame(history_LSTM.history)
# Rename column
metrics_LSTM.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy',
                         'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
def plot_graphs2(var1, var2, string):
    metrics_LSTM[[var1, var2]].plot()
    plt.title('LSTM Model: Training and Validation ' + string)
    plt.xlabel ('Number of epochs')
    plt.ylabel(string)
    plt.legend([var1, var2])
plot_graphs2('Training_Loss', 'Validation_Loss', 'loss')
plot_graphs2('Training_Accuracy', 'Validation_Accuracy', 'accuracy')

In [ ]:
from keras.preprocessing.sequence import pad_sequences

def predict_message(pred_text):
    prediction = []
    new_seq = tokenizer.texts_to_sequences([pred_text])
    padded = pad_sequences(new_seq, maxlen =max_len,
                      padding = padding_type,
                      truncating=trunc_type)
    prediction_prob = model_dense.predict(padded)[0][0]
    prediction.append(prediction_prob)
    if prediction_prob>0.5:
      prediction.append('spam')
    
    else:
      prediction.append('ham')

    return (prediction)


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("Model can predict!")
  else:
    print("Model can't predict!")
  

test_predictions()

JUST THIS!<br>
THANK YOU!<br><br>
~HAMORA HADI